In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load the uploaded data files
production_df = pd.read_csv(r'C:\Repositories\electricity_lstm\data\realised_creation_201501010000_202407010000_15_min.csv', delimiter=';')
consumption_df = pd.read_csv(r'C:\Repositories\electricity_lstm\data\realised_consumption_201501010000_202407010000_15_min.csv', delimiter=';')


In [ ]:
# Check the first few rows of the dataframes
consumption_df.columns

In [ ]:
# Check the columns of the production data
production_df.columns

In [ ]:
#### Convert the columns to a more readable format
# Translate column names for production data
production_df.columns = ['start_time', 'end_time', 'biomass_mwh', 'hydropower_mwh', 'wind_offshore_mwh',
                        'wind_onshore_mwh', 'solar_mwh', 'other_renewables_mwh', 'nuclear_mwh',
                        'lignite_mwh', 'hard_coal_mwh', 'natural_gas_mwh', 'pumped_storage_mwh', 'other_conventional_mwh']

# Translate column names for consumption data
consumption_df.columns = ['start_time', 'end_time', 'total_load_mwh', 'residual_load_mwh', 'saved_pumped_storage_mwh']


In [ ]:
production_df.head()

In [ ]:
#### Convert date and time columns to datetime format
# production data
production_df['start_time'] = pd.to_datetime(production_df['start_time'], format='%d.%m.%Y %H:%M')
production_df['end_time'] = pd.to_datetime(production_df['end_time'], format='%d.%m.%Y %H:%M')

# consumption data
consumption_df['start_time'] = pd.to_datetime(consumption_df['start_time'], format='%d.%m.%Y %H:%M')
consumption_df['end_time'] = pd.to_datetime(consumption_df['end_time'], format='%d.%m.%Y %H:%M')


In [ ]:
production_df['end_time']

In [ ]:
# merge the two dataframes
data = pd.merge(production_df, consumption_df, on=['start_time', 'end_time'])

In [ ]:
data.head()

In [ ]:
# check if there is any NaN values in the data
data.isnull().sum()

In [ ]:
# save the merged data
data.to_csv(r'C:\Repositories\electricity_lstm\data\cleaned_merged_data_with_15_min_interval.csv', index=False)

In [ ]:
data.columns

In [ ]:
data['total_production_mwh'] = data[['biomass_mwh', 'hydropower_mwh', 'wind_offshore_mwh', 'wind_onshore_mwh',
                                    'solar_mwh', 'other_renewables_mwh', 'nuclear_mwh', 'lignite_mwh', 'hard_coal_mwh',
                                    'natural_gas_mwh', 'pumped_storage_mwh', 'other_conventional_mwh']].sum(axis=1)


In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(data['start_time'], data['total_production_mwh'], label='Total Production (MWh)')
plt.plot(data['start_time'], data['total_load_mwh'], label='Total Load (MWh)')
plt.xlabel('Time')
plt.ylabel('MWh')
# plt.title('Energy Production vs. Load')
plt.legend()
plt.grid(True)
plt.show()

In [67]:
# extract granular time information from the date_time in the data
data['hour'] = data["start_time"].dt.hour
data['minute'] = data["start_time"].dt.minute
data['day_of_week'] = data["start_time"].dt.dayofweek
data['day_of_year'] = data["start_time"].dt.dayofyear
data['week_of_year'] = data['start_time'].dt.isocalendar().week

In [68]:
data.head()

,start_time,end_time,biomass_mwh,hydropower_mwh,wind_offshore_mwh,wind_onshore_mwh,solar_mwh,other_renewables_mwh,nuclear_mwh,lignite_mwh,...,other_conventional_mwh,total_load_mwh,residual_load_mwh,saved_pumped_storage_mwh,total_production_mwh,hour,minute,day_of_week,day_of_year,week_of_year
0,2015-01-01 00:00:00,2015-01-01 00:15:00,"1.005,50","288,25","130,00","2.028,25","0,00","33,25","2.685,50","3.964,75",...,"1.235,00","11.210,00","9.051,75","123,00","1.005,50288,25130,002.028,250,0033,252.685,503...",0,0,3,1,1
1,2015-01-01 00:15:00,2015-01-01 00:30:00,"1.007,00","287,75","129,25","2.023,00","0,00","33,25","2.646,25","3.950,75",...,"1.213,75","11.118,75","8.966,50","154,25","1.007,00287,75129,252.023,000,0033,252.646,253...",0,15,3,1,1
2,2015-01-01 00:30:00,2015-01-01 00:45:00,"1.006,50","292,75","128,50","2.040,25","0,00","33,25","2.660,75","3.912,25",...,"1.218,50","11.144,75","8.976,00","160,50","1.006,50292,75128,502.040,250,0033,252.660,753...",0,30,3,1,1
3,2015-01-01 00:45:00,2015-01-01 01:00:00,"1.005,25","289,50","128,75","2.036,50","0,00","33,25","2.718,00","3.859,50",...,"1.242,00","11.126,75","8.961,50","163,75","1.005,25289,50128,752.036,500,0033,252.718,003...",0,45,3,1,1
4,2015-01-01 01:00:00,2015-01-01 01:15:00,"999,00","295,25","128,75","2.045,75","0,00","33,25","2.772,25","3.888,00",...,"1.247,25","11.009,50","8.835,00","87,50","999,00295,25128,752.045,750,0033,252.772,253.8...",1,0,3,1,1


In [69]:
# encode the date and time features in a way that captures the periodicity of the data
# include sine and cosine transformations for the hour and day of the week
def encode_feature(data, column_name:str, max_value:int):
    data[column_name + '_sin'] = np.sin(2 * np.pi * data[column_name] / max_value)
    data[column_name + '_cos'] = np.cos(2 * np.pi * data[column_name] / max_value)
    return data

In [70]:
# capture the cyclical nature of hours in a day.
encode_feature(data, 'hour', 24)
# capture the cyclical nature of minutes within each hour.
encode_feature(data, 'minute', 60)
# capture the cyclical nature of days within a week.
encode_feature (data, 'day_of_week', 7)
# capture the cyclical nature of days within a year.
encode_feature(data,'day_of_year', 365)
# capture the cyclical nature of weeks within a year.
encode_feature(data, 'week_of_year', 52)

,start_time,end_time,biomass_mwh,hydropower_mwh,wind_offshore_mwh,wind_onshore_mwh,solar_mwh,other_renewables_mwh,nuclear_mwh,lignite_mwh,...,hour_sin,hour_cos,minute_sin,minute_cos,day_of_week_sin,day_of_week_cos,day_of_year_sin,day_of_year_cos,week_of_year_sin,week_of_year_cos
0,2015-01-01 00:00:00,2015-01-01 00:15:00,"1.005,50","288,25","130,00","2.028,25","0,00","33,25","2.685,50","3.964,75",...,0.000000,1.000000,0.000000e+00,1.000000e+00,0.433884,-0.900969,0.017213,0.999852,0.120537,0.992709
1,2015-01-01 00:15:00,2015-01-01 00:30:00,"1.007,00","287,75","129,25","2.023,00","0,00","33,25","2.646,25","3.950,75",...,0.000000,1.000000,1.000000e+00,2.832769e-16,0.433884,-0.900969,0.017213,0.999852,0.120537,0.992709
2,2015-01-01 00:30:00,2015-01-01 00:45:00,"1.006,50","292,75","128,50","2.040,25","0,00","33,25","2.660,75","3.912,25",...,0.000000,1.000000,5.665539e-16,-1.000000e+00,0.433884,-0.900969,0.017213,0.999852,0.120537,0.992709
3,2015-01-01 00:45:00,2015-01-01 01:00:00,"1.005,25","289,50","128,75","2.036,50","0,00","33,25","2.718,00","3.859,50",...,0.000000,1.000000,-1.000000e+00,-1.836970e-16,0.433884,-0.900969,0.017213,0.999852,0.120537,0.992709
4,2015-01-01 01:00:00,2015-01-01 01:15:00,"999,00","295,25","128,75","2.045,75","0,00","33,25","2.772,25","3.888,00",...,0.258819,0.965926,0.000000e+00,1.000000e+00,0.433884,-0.900969,0.017213,0.999852,0.120537,0.992709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333087,2024-06-30 22:45:00,2024-06-30 23:00:00,"1.034,00","669,75","337,00","1.400,00","0,00","28,00",-,"2.276,50",...,-0.500000,0.866025,-1.000000e+00,-1.836970e-16,-0.781831,0.623490,0.008607,-0.999963,-0.0,-1.0
333088,2024-06-30 23:00:00,2024-06-30 23:15:00,"1.036,25","643,00","342,25","1.328,25","0,00","28,00",-,"2.290,50",...,-0.258819,0.965926,0.000000e+00,1.000000e+00,-0.781831,0.623490,0.008607,-0.999963,-0.0,-1.0
333089,2024-06-30 23:15:00,2024-06-30 23:30:00,"1.035,00","691,25","330,00","1.283,75","0,00","28,00",-,"2.296,00",...,-0.258819,0.965926,1.000000e+00,2.832769e-16,-0.781831,0.623490,0.008607,-0.999963,-0.0,-1.0
333090,2024-06-30 23:30:00,2024-06-30 23:45:00,"1.035,50","652,00","292,50","1.251,75","0,00","28,00",-,"2.289,00",...,-0.258819,0.965926,5.665539e-16,-1.000000e+00,-0.781831,0.623490,0.008607,-0.999963,-0.0,-1.0


In [ ]:


# Extract date features from the start_time column in the consumption data
consumption_df['day_of_week'] = consumption_df['start_time'].dt.dayofweek
consumption_df['day_of_month'] = consumption_df['start_time'].dt.day
consumption_df['month'] = consumption_df['start_time'].dt.month
consumption_df['day_of_year'] = consumption_df['start_time'].dt.dayofyear

# Sine and Cosine transformation for cyclic features
consumption_df['day_of_year_sin'] = np.sin(2 * np.pi * consumption_df['day_of_year'] / 365.25)
consumption_df['day_of_year_cos'] = np.cos(2 * np.pi * consumption_df['day_of_year'] / 365.25)

# Drop the original 'day_of_year' column
consumption_df.drop(columns=['day_of_year'], inplace=True)

# Repeat the same for the production data
production_df['day_of_week'] = production_df['start_time'].dt.dayofweek
production_df['day_of_month'] = production_df['start_time'].dt.day
production_df['month'] = production_df['start_time'].dt.month
production_df['day_of_year'] = production_df['start_time'].dt.dayofyear

production_df['day_of_year_sin'] = np.sin(2 * np.pi * production_df['day_of_year'] / 365.25)
production_df['day_of_year_cos'] = np.cos(2 * np.pi * production_df['day_of_year'] / 365.25)

production_df.drop(columns=['day_of_year'], inplace=True)

# Merge production and consumption data on the start_time column
data = pd.merge(production_df, consumption_df, on='start_time')

# Display the first few rows of the merged data
print("Merged Data with Date Features:")
print(data.head())

In [ ]:
data = pd.merge(production_df, consumption_df, on='time')